In [0]:
#Import
import pandas as pd
import numpy as np

import json
from pandas.io.json import json_normalize
import networkx as nx

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df = pd.read_json('/content/drive/My Drive/dblp-ref-0.json',lines=True)

In [0]:
df[1]

,abstract,authors,id,n_citation,references,title,venue,year
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,"[8c78e4b0-632b-4293-b491-85b1976675e6, 9cdc54f...",Development of Remote Monitoring and Control D...,,2011
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998


In [0]:
df.head(1)

,abstract,authors,id,n_citation,references,title,venue,year
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013


In [0]:
pip install git+https://github.com/OrganicIrradiation/scholarly.git

In [0]:
import scholarly

print(next(scholarly.search_author('Steven A. Cholewiak')))

In [0]:
df_copy = df.copy()

In [0]:
def calculate_page_rank_utility(df_pageRank):
    from  itertools import chain
    df_pageRank_flatten = pd.DataFrame({
            "abstract": np.repeat(df_pageRank.abstract.values, df_pageRank.references.str.len()),
            "authors": np.repeat(df_pageRank.authors.values, df_pageRank.references.str.len()),
            "id": np.repeat(df_pageRank.id.values, df_pageRank.references.str.len()),
            "n_citation": np.repeat(df_pageRank.n_citation.values, df_pageRank.references.str.len()),
            "references": list(chain.from_iterable(df_pageRank.references)),
            "title": np.repeat(df_pageRank.title.values, df_pageRank.references.str.len()),
            "venue": np.repeat(df_pageRank.venue.values, df_pageRank.references.str.len()),
            "year": np.repeat(df_pageRank.year.values, df_pageRank.references.str.len()),        
            "pageRankValue": np.repeat(df_pageRank.pageRankValue.values, df_pageRank.references.str.len()),
            "totalOutbounds": np.repeat(df_pageRank.totalOutbounds.values, df_pageRank.references.str.len()),
            "ratio": np.repeat(df_pageRank.ratio.values, df_pageRank.references.str.len())        
    })

    df_pageRank_flatten_groupby = df_pageRank_flatten.groupby('references').agg({'ratio' : 'sum'})
    df_pageRank_flatten_groupby['id'] = df_pageRank_flatten_groupby.index
    df_pageRank_flatten_groupby['newPageRankValue'] = df_pageRank_flatten_groupby['ratio']

    df_pageRank_new = pd.merge(df_pageRank, df_pageRank_flatten_groupby[['id', 'newPageRankValue']], how='left', on='id')
    df_pageRank_new.pageRankValue = df_pageRank_new.newPageRankValue
    df_pageRank_new['ratio'] = df_pageRank_new.pageRankValue / df_pageRank_new.totalOutbounds
    df_pageRank_new = df_pageRank_new.drop(columns = ['newPageRankValue'])

    return df_pageRank_new

def calculate_page_rank (df, iteration):
    for i in range (0, iteration):
        df = calculate_page_rank_utility(df)
    return df

In [0]:
df_page_rank_actual = df.copy()

In [0]:
df_page_rank_actual.shape

(1000000, 8)

In [0]:
#placing [self] where references is nan
df_page_rank_actual.loc[df_page_rank_actual['references'].isnull(),['references']] = df_page_rank_actual.loc[df_page_rank_actual['references'].isnull(),'references'].apply(lambda references : ['self'])

#also do for empty array
df_page_rank_actual.references = df_page_rank_actual.references.apply(lambda y: ['self'] if len(y)==0 else y)

#without [self], empty array, on flatten entries with empty array will be deleted

In [0]:
from  itertools import chain
df_page_rank_actual_flatten_references = pd.DataFrame({
            "id": np.repeat(df_page_rank_actual.id.values, df_page_rank_actual.references.str.len()),
            "references": list(chain.from_iterable(df_page_rank_actual.references))
    })

In [0]:
df_page_rank_actual_flatten_references.shape

(7647266, 2)

In [0]:
df_page_rank_actual_flatten_references.references.unique().shape

(1291817,)

In [0]:
unique_paper_id = df_page_rank_actual_flatten_references.id.unique()

unique_reference_id = df_page_rank_actual_flatten_references.references.unique()

print (unique_paper_id.shape)

print (unique_reference_id.shape)

diff_set = set(unique_reference_id)-set(unique_paper_id)

(1000000,)
(1291817,)


In [0]:
len(diff_set)

854174

In [0]:
df_page_rank_actual_flatten_references = df_page_rank_actual_flatten_references[~df_page_rank_actual_flatten_references['references'].isin(diff_set)]

In [0]:
df_page_rank_actual_flatten_references.shape

(2253220, 3)

In [0]:
print (df_page_rank_actual_flatten_references.references.unique().shape)

print (df_page_rank_actual_flatten_references.shape)

(437643,)
(2253220, 2)


In [0]:
df_page_rank_actual_flatten_references.head(5)

,id,references,reach
4,001c58d3-26ad-46b3-ab3a-c1e557d16821,3a926fef-7422-4654-8776-8e31b45be563,0
6,001c58d3-26ad-46b3-ab3a-c1e557d16821,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,0
15,001c8744-73c4-4b04-9364-22d31a10dbf1,2d84c0f2-e656-4ce7-b018-90eda1c132fe,0
16,001c8744-73c4-4b04-9364-22d31a10dbf1,a083a1b9-8dfb-45d6-99a9-fa30c4a6e9f5,0
17,00338203-9eb3-40c5-9f31-cbac73a519ec,8c78e4b0-632b-4293-b491-85b1976675e6,0


In [0]:
df_page_rank_actual_flatten_references[df_page_rank_actual_flatten_references.id=='6f52f995-7c4c-4a92-83aa-d1c9fbd2465c']

,id,references,reach
2482479,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,2bbcd656-eb94-49a1-b533-bff1fab0abab,0
2482481,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,5b5b1666-eeb3-4f7f-8183-cb19ae35097e,0
2482483,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,c42df285-114c-484a-8e08-30faba0852d9,0


In [0]:
df_page_rank_actual_flatten_references.to_pickle("/content/drive/My Drive/df_page_rank_actual_flatten_references.pkl")

In [0]:
df_page_rank_actual_flatten_references = pd.read_pickle("/content/drive/My Drive/df_page_rank_actual_flatten_references.pkl")
df_page_rank_actual_flatten_references['reach'] = 0

In [0]:
df_page_rank_actual_flatten_references[df_page_rank_actual_flatten_references.id=='6f52f995-7c4c-4a92-83aa-d1c9fbd2465c']

,id,references,reach
2482479,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,2bbcd656-eb94-49a1-b533-bff1fab0abab,0
2482481,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,5b5b1666-eeb3-4f7f-8183-cb19ae35097e,0
2482483,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,c42df285-114c-484a-8e08-30faba0852d9,0


In [0]:
df_page_rank_actual_flatten_references.shape

(2253220, 3)

In [0]:
df_page_rank_actual_flatten_references_small = df_page_rank_actual_flatten_references[0:1000]

In [0]:
df_page_rank_actual_flatten_references_small.shape

(1000, 3)

In [0]:
df_page_rank_actual_flatten_references_small.head(5)

,id,references,reach
4,001c58d3-26ad-46b3-ab3a-c1e557d16821,3a926fef-7422-4654-8776-8e31b45be563,0
6,001c58d3-26ad-46b3-ab3a-c1e557d16821,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,0
15,001c8744-73c4-4b04-9364-22d31a10dbf1,2d84c0f2-e656-4ce7-b018-90eda1c132fe,0
16,001c8744-73c4-4b04-9364-22d31a10dbf1,a083a1b9-8dfb-45d6-99a9-fa30c4a6e9f5,0
17,00338203-9eb3-40c5-9f31-cbac73a519ec,8c78e4b0-632b-4293-b491-85b1976675e6,0


In [0]:
author_graph = nx.Graph() 

In [0]:
def update_author_graph(coauthors):  
  edges=[]
  for author1 in coauthors:
    for author2 in coauthors:
      if author1 != author2:        
        edge = (author1,author2)
        edges.append(edge)
  author_graph.add_edges_from(edges)
  return coauthors

def max_collaboration_distance(author_set_1,author_set_2):
  farthestAuthorDist = 0
  for author1 in author_set_1:
    for author2 in author_set_2:
      try:
        #print ("type of author1:",type(author1))
        dist = nx.shortest_path_length(author_graph, source=author1, target=author2)
      except nx.NetworkXNoPath:
        dist = float("inf")
        #print ("No path exists between "+author1+" and "+author2)
        pass
      except nx.NodeNotFound:
        dist = 0
        #print ("Node not found "+author1+" or "+author2)
        pass
      #print ("Distance between "+author1 +" and "+ author2 +" is : "+ str(dist))
      farthestAuthorDist = max(farthestAuthorDist, dist)
  return farthestAuthorDist

In [0]:
df.shape

(1000000, 8)

In [0]:
train_df_authors_small = df[0:1000000]
[x['authors'].apply(update_author_graph) for x in np.split(train_df_authors_small, np.arange(5, len(train_df_authors_small), 5))]

[0    [Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...
 1                          [Gareth Beale, Graeme Earl]
 2    [Altaf Hossain, Faisal Zaman, Mohammed Nasser,...
 3    [Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...
 4                  [Giovanna Guerrini, Isabella Merlo]
 Name: authors, dtype: object,
 5    [Rafael Álvarez, Leandro Tortosa, José-Francis...
 6                 [Jovan Dj. Golic, Guglielmo Morgari]
 7     [Guzin Ulutas, Mustafa Ulutas, Vasif V. Nabiyev]
 8                 [Pranay Chaudhuri, Hussein Thompson]
 9                 [Dominik Szajerman, Adam Jurczyński]
 Name: authors, dtype: object,
 10    [Ankita Brahmachari, Priya Singh, Avdhesh Garg...
 11    [David Al-Dabass, Christopher Goodwin, K. Siva...
 12              [Alvaro L. Islas, Constance M. Schober]
 13                      [Patrick Cousot, Radhia Cousot]
 14              [Minoru Shigenaga, Yoshihiro Sekiguchi]
 Name: authors, dtype: object,
 15         [Reiko Heckel, Tom Mens, Michel Wermelinger]
 16  

In [0]:
nx.write_gpickle(author_graph, "/content/drive/My Drive/author_graph.gpickle")

In [0]:
author_graph = nx.read_gpickle("/content/drive/My Drive/author_graph.gpickle")

In [0]:
def update_max_score(row):  
  
  authors1_series =  df[df.id==row['references']]['authors']
  
  #print ("\n Paper id : " + row['references'])
  author_set_1 = get_list_of_authors(authors1_series)
  #print (" Authors : ",author_set_1)
  
  
  authors2_series = df[df.id==row['id']]['authors']
  #print (" Citation id : ",row['id'])
  author_set_2 = get_list_of_authors(authors2_series)
  #print (" Authors : ", author_set_2)
  
  #print ("Reach :",max_collaboration_distance(author_set_1,author_set_2))
  
  row['reach'] = max_collaboration_distance(author_set_1,author_set_2)
    
  return row
  
  
#This function takes a object of type Series and converts it to a list of authors
def get_list_of_authors(author_series):
  authors = []
  for key,value in author_series.iteritems():
    for author in value:
      authors.append(author)    
  return authors  

In [0]:
df_page_rank_actual_flatten_references.shape

(2253220, 3)

In [0]:
df_page_rank_actual_flatten_references.head(5)

,id,references,reach
4,001c58d3-26ad-46b3-ab3a-c1e557d16821,3a926fef-7422-4654-8776-8e31b45be563,0
6,001c58d3-26ad-46b3-ab3a-c1e557d16821,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,0
15,001c8744-73c4-4b04-9364-22d31a10dbf1,2d84c0f2-e656-4ce7-b018-90eda1c132fe,0
16,001c8744-73c4-4b04-9364-22d31a10dbf1,a083a1b9-8dfb-45d6-99a9-fa30c4a6e9f5,0
17,00338203-9eb3-40c5-9f31-cbac73a519ec,8c78e4b0-632b-4293-b491-85b1976675e6,0


In [0]:
low=112700
high=112800
for i in range(200):
  print (str(low)+", "+str(high))
  #df_page_rank_actual_flatten_references_small = df_page_rank_actual_flatten_references[low:high]
  df_page_rank_actual_flatten_references_small = df_page_rank_actual_flatten_references[low:high]
  reach_df = df_page_rank_actual_flatten_references_small.apply(update_max_score, axis=1)
  reach_df_compressed = reach_df[['id','reach']]
  max_reach_df_compressed = reach_df_compressed.groupby('id',as_index=False)['reach'].max()
  filename = "/content/drive/My Drive/author_graph/max_reach_df_compressed_" + str(low)+"_"+str(high)+".pkl"
  print ("will save df to:", filename)
  max_reach_df_compressed.to_pickle(filename)
  low+=100
  high+=100

112700, 112800
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_112700_112800.pkl
112800, 112900
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_112800_112900.pkl
112900, 113000
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_112900_113000.pkl
113000, 113100
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_113000_113100.pkl
113100, 113200
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_113100_113200.pkl
113200, 113300
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_113200_113300.pkl
113300, 113400
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_113300_113400.pkl
113400, 113500
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_113400_113500.pkl
113500, 113600
will save df to: /content/drive/My Drive/author_graph/max_reach_df_compressed_113500_113600.pkl
1

KeyboardInterrupt: ignored

In [0]:
import os
os.listdir("/content/drive/My Drive/author_graph/")

['max_reach_df_compressed_27600_27700.pkl',
 'max_reach_df_compressed_27700_27800.pkl',
 'max_reach_df_compressed_27800_27900.pkl',
 'max_reach_df_compressed_27900_28000.pkl',
 'max_reach_df_compressed_28000_28100.pkl',
 'max_reach_df_compressed_28100_28200.pkl',
 'max_reach_df_compressed_28200_28300.pkl',
 'max_reach_df_compressed_28300_28400.pkl',
 'max_reach_df_compressed_28400_28500.pkl',
 'max_reach_df_compressed_28500_28600.pkl',
 'max_reach_df_compressed_28600_28700.pkl',
 'max_reach_df_compressed_28700_28800.pkl',
 'max_reach_df_compressed_28800_28900.pkl',
 'max_reach_df_compressed_28900_29000.pkl',
 'max_reach_df_compressed_29000_29100.pkl',
 'max_reach_df_compressed_29100_29200.pkl',
 'max_reach_df_compressed_29200_29300.pkl',
 'max_reach_df_compressed_29300_29400.pkl',
 'max_reach_df_compressed_29400_29500.pkl',
 'max_reach_df_compressed_29500_29600.pkl',
 'max_reach_df_compressed_29600_29700.pkl',
 'max_reach_df_compressed_29700_29800.pkl',
 'max_reach_df_compressed_29800_

In [0]:
dirs = os.listdir("/content/drive/My Drive/author_graph/")

In [0]:
df_list=[]
for file in dirs:
  #print (file)
  #print (type(file))
  reach_df = pd.read_pickle("/content/drive/My Drive/author_graph/"+file)
  #print (reach_df.shape)
  df_list.append(reach_df)
#print (df_list)

In [0]:
bigdf = pd.concat(df_list)

In [0]:
bigdf.shape

(35480, 2)

In [0]:
bigdf.head(5)

,id,reach
0,403596ee-98c2-47b3-9490-b482daa8c6c6,6.000000
1,403cbeb3-d1e6-45d6-8f4c-d5f8ebd4ebe9,6.000000
2,411b8486-936e-4cdd-a6ff-57908c89b6e4,6.000000
3,4127087c-c096-4c1c-b4fa-2a0d15743f79,5.000000
4,414d8e55-e473-407d-bdfe-c122d4da4dac,inf


In [0]:
df_page_rank_actual_flatten_references_small.shape

(1000, 3)

In [0]:
df_page_rank_actual_flatten_references_small = df_page_rank_actual_flatten_references[1000:2000]

In [0]:
reach_df = df_page_rank_actual_flatten_references_small.apply(update_max_score, axis=1)

In [0]:
df_page_rank_actual_flatten_references_small.head(5)

,id,references,reach
4,001c58d3-26ad-46b3-ab3a-c1e557d16821,3a926fef-7422-4654-8776-8e31b45be563,0
6,001c58d3-26ad-46b3-ab3a-c1e557d16821,6f52f995-7c4c-4a92-83aa-d1c9fbd2465c,0
15,001c8744-73c4-4b04-9364-22d31a10dbf1,2d84c0f2-e656-4ce7-b018-90eda1c132fe,0
16,001c8744-73c4-4b04-9364-22d31a10dbf1,a083a1b9-8dfb-45d6-99a9-fa30c4a6e9f5,0
17,00338203-9eb3-40c5-9f31-cbac73a519ec,8c78e4b0-632b-4293-b491-85b1976675e6,0


In [0]:
reach_df.shape

(1000, 3)

In [0]:
reach_df.head(5)

,id,references,reach
2858,249589e6-5d60-429a-933b-3e5f52f4f79f,99b6d948-83ec-41aa-8c83-8d20cf4e713e,4.0
2859,249589e6-5d60-429a-933b-3e5f52f4f79f,ad413a89-91bb-47fb-94b3-a94e3d7baef4,4.0
2865,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,020410d3-9cc9-4af2-b0ab-9ee0cbea3388,0.0
2866,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,1c432e1f-725c-4bb9-bdd7-714cebc02b29,6.0
2867,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,2a3defdb-a33c-422b-99e4-e2cfd04467f5,5.0


In [0]:
reach_df_compressed = reach_df[['id','reach']]

In [0]:
reach_df_compressed.head(5)

,id,reach
2858,249589e6-5d60-429a-933b-3e5f52f4f79f,4.0
2859,249589e6-5d60-429a-933b-3e5f52f4f79f,4.0
2865,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,0.0
2866,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,6.0
2867,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,5.0


In [0]:
max_reach_df_compressed = reach_df_compressed.groupby('id',as_index=False)['reach'].max()

In [0]:
max_reach_df_compressed.shape

(277, 2)

In [0]:
max_reach_df_compressed.head(5)

,id,reach
0,249589e6-5d60-429a-933b-3e5f52f4f79f,4.0
1,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,6.0
2,24d0d145-133c-41ab-a0a5-9dcb8bc4f925,6.0
3,25131170-197c-40a9-8cb1-047118078ab0,4.0
4,25377e07-5400-4f78-8e01-4abf4fc0172e,6.0


In [0]:
max_reach_df_compressed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 276
Data columns (total 2 columns):
id       277 non-null object
reach    277 non-null float64
dtypes: float64(1), object(1)
memory usage: 6.5+ KB


In [0]:
max_reach_df_compressed.to_pickle("/content/drive/My Drive/max_reach_df_compressed_2.pkl")

In [0]:
max_reach_df_compressed_1 = pd.read_pickle("/content/drive/My Drive/max_reach_df_compressed_1.pkl")
max_reach_df_compressed_2 = pd.read_pickle("/content/drive/My Drive/max_reach_df_compressed_2.pkl")

In [0]:
bigdf = pd.concat([max_reach_df_compressed_1,max_reach_df_compressed_2])

In [0]:
bigdf.shape

(35480, 2)

In [0]:
bigdf.to_pickle("/content/drive/My Drive/bigdf_1.pkl")

In [0]:
print (len(bigdf['id'].unique()))

34516


In [0]:
bigdf.groupby('reach').count().sort_values(by=['id'],ascending=False)

,id
reach,
5.000000,9554
6.000000,7509
4.000000,5460
7.000000,3211
2.000000,2350
3.000000,1943
inf,1720
1.000000,1455
8.000000,1014


In [0]:
bigdf.head(5)

,id,reach
0,001c58d3-26ad-46b3-ab3a-c1e557d16821,6.0
1,001c8744-73c4-4b04-9364-22d31a10dbf1,7.0
2,00338203-9eb3-40c5-9f31-cbac73a519ec,6.0
3,00a119c4-d367-4607-b3c8-b237f2971bff,6.0
4,00c85316-bddf-4bcb-93f5-097adadd73c2,0.0


In [0]:
bigdf_grouped_by_id = bigdf.groupby('id',as_index=False)['reach'].max()

In [0]:
bigdf_grouped_by_id.shape

(541, 2)

In [0]:
bigdf_grouped_by_id.head(5)

,id,reach
0,001c58d3-26ad-46b3-ab3a-c1e557d16821,6.0
1,001c8744-73c4-4b04-9364-22d31a10dbf1,7.0
2,00338203-9eb3-40c5-9f31-cbac73a519ec,6.0
3,00a119c4-d367-4607-b3c8-b237f2971bff,6.0
4,00c85316-bddf-4bcb-93f5-097adadd73c2,0.0


In [0]:
max_reach_df_compressed_2.head(5)

,id,reach
0,249589e6-5d60-429a-933b-3e5f52f4f79f,4.0
1,24c9aff2-6438-4fbe-acf1-112b64aaf1cd,6.0
2,24d0d145-133c-41ab-a0a5-9dcb8bc4f925,6.0
3,25131170-197c-40a9-8cb1-047118078ab0,4.0
4,25377e07-5400-4f78-8e01-4abf4fc0172e,6.0


In [0]:
reach_df_grouped_by_id_series = reach_df_compressed.groupby(['id'])['reach'].max()

In [0]:
reach_df_grouped_by_id_series.shape

(3,)

In [0]:
reach_df_grouped_by_id_series

id
001c58d3-26ad-46b3-ab3a-c1e557d16821    6
001c8744-73c4-4b04-9364-22d31a10dbf1    7
00338203-9eb3-40c5-9f31-cbac73a519ec    6
Name: reach, dtype: int64

In [0]:
print (type(reach_df_grouped_by_id_series))

<class 'pandas.core.series.Series'>


In [0]:
reach_df_grouped_by_id = reach_df_grouped_by_id_series.to_frame()

In [0]:
print (type(reach_df_grouped_by_id))

<class 'pandas.core.frame.DataFrame'>


In [0]:
reach_df_grouped_by_id.shape

(3, 1)

In [0]:
reach_df_grouped_by_id.head(5)

,reach
id,
001c58d3-26ad-46b3-ab3a-c1e557d16821,6
001c8744-73c4-4b04-9364-22d31a10dbf1,7
00338203-9eb3-40c5-9f31-cbac73a519ec,6


In [0]:
for i, v in reach_df_grouped_by_id_series.iteritems():
    print('index: ', i, 'value: ', v)
    #print (type(i))
    #print (type(v))
    reach_df_grouped_by_id[]

index:  001c58d3-26ad-46b3-ab3a-c1e557d16821 value:  6
<class 'str'>
<class 'int'>
index:  001c8744-73c4-4b04-9364-22d31a10dbf1 value:  7
<class 'str'>
<class 'int'>
index:  00338203-9eb3-40c5-9f31-cbac73a519ec value:  6
<class 'str'>
<class 'int'>


In [0]:
nx.draw(author_graph,node_size=100,with_labels=False)

In [0]:
author_series = df[df.id=='3a926fef-7422-4654-8776-8e31b45be563']['authors']
print (get_list_of_authors(author_series))
list = ['Veronica Sundstedt', 'Alan Chalmers', 'Philippe Martinez']
print (list)

['Veronica Sundstedt', 'Alan Chalmers', 'Philippe Martinez']
['Veronica Sundstedt', 'Alan Chalmers', 'Philippe Martinez']


In [0]:
df_page_rank_actual_flatten_references = df_page_rank_actual_flatten_references.groupby('id').agg({'references':lambda x: list(x)})

In [0]:
df_page_rank_actual_flatten_references.shape

(629496, 1)

In [0]:
df_page_rank_actual_flatten_references['id'] = df_page_rank_actual_flatten_references.index

In [0]:
df_page_rank_actual_flatten_references['references2'] = df_page_rank_actual_flatten_references['references']

In [0]:
df_page_rank_actual_flatten_references.shape

(629496, 3)

In [0]:
df_page_rank_actual = pd.merge(df_page_rank_actual, df_page_rank_actual_flatten_references[['id', 'references2']], how='left', on='id')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
df_page_rank_actual.shape

(1000000, 9)

In [0]:
df_page_rank_actual['references'] = df_page_rank_actual['references2']

In [0]:
df_page_rank_actual = df_page_rank_actual.drop(columns = ['references2'])

In [0]:
df_page_rank_actual.shape

(1000000, 8)

In [0]:
#add 3 columns required for calculating page rank

df_page_rank_actual['pageRankValue'] = 100
df_page_rank_actual['totalOutbounds'] = df_page_rank_actual.references.str.len()
df_page_rank_actual['ratio'] = df_page_rank_actual.pageRankValue / df_page_rank_actual.totalOutbounds


In [0]:
df_page_rank_actual.shape

(1000000, 11)

In [0]:
df_page_rank_actual['pageRankValue'].sum()

100000000

In [0]:
df_page_rank_actual_iterate1 = calculate_page_rank(df_page_rank_actual, 1)
#error because of NaN in references data .... after left join .... some rows were not found

In [0]:
df_page_rank_actual.head()

,abstract,authors,id,n_citation,references,title,venue,year,pageRankValue,totalOutbounds,ratio
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,NaN,Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013,100,NaN,NaN
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[3a926fef-7422-4654-8776-8e31b45be563, 6f52f99...",A methodology for the physically accurate visu...,visual analytics science and technology,2011,100,2.0,50.0
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009,100,2.0,50.0
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,[8c78e4b0-632b-4293-b491-85b1976675e6],Development of Remote Monitoring and Control D...,,2011,100,1.0,100.0
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998,100,NaN,NaN


In [0]:
#placing [self] where references is nan
df_page_rank_actual.loc[df_page_rank_actual['references'].isnull(),['references']] = df_page_rank_actual.loc[df_page_rank_actual['references'].isnull(),'references'].apply(lambda references : ['self'])

#also do for empty array
df_page_rank_actual.references = df_page_rank_actual.references.apply(lambda y: ['self'] if len(y)==0 else y)

#without [self], empty array, on flatten entries with empty array will be deleted

In [0]:
df_page_rank_actual.head()

,abstract,authors,id,n_citation,references,title,venue,year,pageRankValue,totalOutbounds,ratio
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,[self],Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013,100,NaN,NaN
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[3a926fef-7422-4654-8776-8e31b45be563, 6f52f99...",A methodology for the physically accurate visu...,visual analytics science and technology,2011,100,2.0,50.0
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009,100,2.0,50.0
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,[8c78e4b0-632b-4293-b491-85b1976675e6],Development of Remote Monitoring and Control D...,,2011,100,1.0,100.0
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,[self],Reasonig about Set-Oriented Methods in Object ...,,1998,100,NaN,NaN


In [0]:
df_page_rank_actual_iterate1 = calculate_page_rank(df_page_rank_actual, 1)

In [0]:
df_page_rank_actual_iterate1.shape

(1000000, 11)

In [0]:
df_page_rank_actual_iterate1['pageRankValue'].sum()

62949600.00000001

In [0]:
df_page_rank_actual_iterate1.head()

,abstract,authors,id,n_citation,references,title,venue,year,pageRankValue,totalOutbounds,ratio
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,[self],Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013,NaN,NaN,NaN
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[3a926fef-7422-4654-8776-8e31b45be563, 6f52f99...",A methodology for the physically accurate visu...,visual analytics science and technology,2011,NaN,2.0,NaN
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009,33.333333,2.0,16.666667
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,[8c78e4b0-632b-4293-b491-85b1976675e6],Development of Remote Monitoring and Control D...,,2011,NaN,1.0,NaN
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,[self],Reasonig about Set-Oriented Methods in Object ...,,1998,NaN,NaN,NaN


In [0]:
df_page_rank_actual_iterate2 = calculate_page_rank(df_page_rank_actual_iterate1, 1)

In [0]:
df_page_rank_actual_iterate2['pageRankValue'].sum()

44888464.56189278

In [0]:
df_page_rank_actual = df_copy.copy()


In [0]:
df_page_rank_actual['references'].isnull().count()

1000000

In [0]:
df_page_rank_actual['references'].count()

861480

In [0]:
df_page_rank_actual.shape

(1000000, 8)